In [1]:
import yfinance as yf
import psycopg2
from psycopg2 import Error
import pandas as pd
from datetime import datetime

# Step 1: Download SPY data from Yahoo Finance
def download_spy_data():
    # Define the ticker symbol for SPY
    ticker = "SPY"
    start_date = "2020-03-24"  
    end_date = "2025-03-24"  
    
    # Fetch data using yfinance
    spy = yf.download(ticker, start=start_date, end=end_date, interval="1d")
    
    # Reset index to make 'Date' a column
    spy = spy.reset_index()
    spy.columns = ['date', 'open', 'high', 'low', 'close', 'adj_close', 'volume']

    #Converting date to datetime for postgreSQL compatibility
    spy['date'] = pd.to_datetime(spy['date']).dt.date
    
    # Save to CSV (optional, for backup)
    spy.to_csv("spy_data.csv", index=False)
    
    return spy

# Step 2: Connect to PostgreSQL and import data
def import_to_postgres(df):
    # Database connection parameters (update these with your own)
    db_params = {
        "host": "localhost",
        "database": "stocks_db",
        "user": "postgres", 
        "password": "password",  
        "port": "5432"  
    }
    
    try:
        # Establish connection
        connection = psycopg2.connect(**db_params)
        cursor = connection.cursor()
        
        # Create table for SPY data
        create_table_query = """
        DROP TABLE IF EXISTS spy_prices;
        CREATE TABLE spy_prices (
            date DATE PRIMARY KEY,
            open NUMERIC,
            high NUMERIC,
            low NUMERIC,
            close NUMERIC,
            adj_close NUMERIC,
            volume BIGINT
        );
        """
        cursor.execute(create_table_query)
        connection.commit()
        print("Table 'spy_prices' created successfully.")
        
        # Insert data into the table
        for index, row in df.iterrows():
            insert_query = """
            INSERT INTO spy_prices (date, open, high, low, close, adj_close, volume)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
            ON CONFLICT (date) DO NOTHING;
            """
            cursor.execute(insert_query, (
                row['date'], row['open'], row['high'], row['low'],
                row['close'], row['adj_close'], row['volume']
            ))
        
        connection.commit()
        print("SPY data imported successfully into PostgreSQL.")
    
    except (Exception, Error) as error:
        print("Error while connecting to PostgreSQL:", error)
    
    finally:
        if connection:
            cursor.close()
            connection.close()
            print("PostgreSQL connection closed.")

# Main execution
if __name__ == "__main__":
    # Download SPY data
    spy_data = download_spy_data()
    print("SPY data downloaded successfully:")
    print(spy_data.head())  # Preview the data
    
    # Import to PostgreSQL
    import_to_postgres(spy_data)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


ValueError: Length mismatch: Expected axis has 6 elements, new values have 7 elements

In [9]:
import yfinance as yf
import psycopg2
from psycopg2 import Error
import pandas as pd
from datetime import datetime

# Step 1: Download SPY data from Yahoo Finance
def download_spy_data():
    ticker = "SPY"
    start_date = "2020-03-24"  
    end_date = "2025-03-24"   
    
    # Fetch data with adjusted close explicitly enabled
    spy = yf.download(ticker, start=start_date, end=end_date, interval="1d", auto_adjust=False)
    
    # Print initial columns for debugging
    print("Initial columns from yfinance:", spy.columns)
    print("Initial column count:", len(spy.columns))
    
    # Reset index to make 'Date' a column
    spy = spy.reset_index()
    
    # Print columns after reset_index
    print("Columns after reset_index:", spy.columns)
    print("Column count after reset_index:", len(spy.columns))
    
    # Expected columns: Date, Open, High, Low, Close, Adj Close, Volume (7 total)
    expected_columns = ['date', 'open', 'high', 'low', 'close', 'adj_close', 'volume']
    
    # Assign column names if we have all 7
    if len(spy.columns) == 7:
        spy.columns = expected_columns
    elif len(spy.columns) == 6:
        # If Adj Close is missing, adjust accordingly
        spy.columns = ['date', 'open', 'high', 'low', 'close', 'volume']
        print("Warning: 'Adj Close' missing from data. Proceeding with 6 columns.")
    else:
        raise ValueError(f"Unexpected column count: {len(spy.columns)}. Columns: {spy.columns}")
    
    # Convert 'date' to datetime.date for PostgreSQL
    spy['date'] = pd.to_datetime(spy['date']).dt.date
    
    # Save to CSV (optional)
    spy.to_csv("spy_data.csv", index=False)
    
    return spy

In [11]:
# Step 2: Connect to PostgreSQL and import data
def import_to_postgres(df):
    
    db_params = {
        "host": "localhost",
        "database": "stocks_db",
        "user": "postgres",  
        "password": "password",  
        "port": "5432"
    }
    
    try:
        # Establish connection
        connection = psycopg2.connect(**db_params)
        cursor = connection.cursor()
        
        # Create table (adjust based on available columns)
        if 'adj_close' in df.columns:
            create_table_query = """
            DROP TABLE IF EXISTS spy_prices;
            CREATE TABLE spy_prices (
                date DATE PRIMARY KEY,
                open NUMERIC,
                high NUMERIC,
                low NUMERIC,
                close NUMERIC,
                adj_close NUMERIC,
                volume BIGINT
            );
            """
            columns = ['date', 'open', 'high', 'low', 'close', 'adj_close', 'volume']
        else:
            create_table_query = """
            DROP TABLE IF EXISTS spy_prices;
            CREATE TABLE spy_prices (
                date DATE PRIMARY KEY,
                open NUMERIC,
                high NUMERIC,
                low NUMERIC,
                close NUMERIC,
                volume BIGINT
            );
            """
            columns = ['date', 'open', 'high', 'low', 'close', 'volume']
        
        cursor.execute(create_table_query)
        connection.commit()
        print("Table 'spy_prices' created successfully.")
        
        # Insert data
        for index, row in df.iterrows():
            insert_query = f"""
            INSERT INTO spy_prices ({', '.join(columns)})
            VALUES ({', '.join(['%s'] * len(columns))})
            ON CONFLICT (date) DO NOTHING;
            """
            cursor.execute(insert_query, tuple(row[col] for col in columns))
        
        connection.commit()
        print("SPY data imported successfully into PostgreSQL.")
    
    except (Exception, Error) as error:
        print("Error while connecting to PostgreSQL:", error)
    
    finally:
        if connection:
            cursor.close()
            connection.close()
            print("PostgreSQL connection closed.")

# Main execution
if __name__ == "__main__":
    # Download SPY data
    spy_data = download_spy_data()
    print("SPY data downloaded successfully:")
    print(spy_data.head())  # Preview the data
    
    # Import to PostgreSQL
    import_to_postgres(spy_data)

[*********************100%***********************]  1 of 1 completed


Initial columns from yfinance: MultiIndex([('Adj Close', 'SPY'),
            (    'Close', 'SPY'),
            (     'High', 'SPY'),
            (      'Low', 'SPY'),
            (     'Open', 'SPY'),
            (   'Volume', 'SPY')],
           names=['Price', 'Ticker'])
Initial column count: 6
Columns after reset_index: MultiIndex([(     'Date',    ''),
            ('Adj Close', 'SPY'),
            (    'Close', 'SPY'),
            (     'High', 'SPY'),
            (      'Low', 'SPY'),
            (     'Open', 'SPY'),
            (   'Volume', 'SPY')],
           names=['Price', 'Ticker'])
Column count after reset_index: 7
SPY data downloaded successfully:
         date        open        high         low       close   adj_close  \
0  2020-03-24  226.075821  243.149994  244.100006  233.800003  234.419998   
1  2020-03-25  229.460236  246.789993  256.350006  239.750000  244.869995   
2  2020-03-26  242.858292  261.200012  262.799988  249.050003  249.520004   
3  2020-03-27  235.624